# Homework #2

## Подготовка

### Устанвока, импорты

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [63]:
import re
import torch
import numpy as np
import random

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

### Установка random state

In [64]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

## Загрузка датасета

Датасет содержит тексты песен Оксимирона. Набор  текстов взят с genius.com
https://huggingface.co/datasets/huggingartists/oxxxymiron

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

dataset = load_dataset("huggingartists/oxxxymiron")

In [67]:
dataset["train"]["text"][1]

'Эй\nScady\nЙо\nДон ли, Волга ли течёт — котомку на плечо\nБоль в груди — там тайничок, открытый фомкой, не ключом\nСколько миль ещё? Перелет короткий был не в счёт\nДолгий пыльный чёс, фургон набит коробками с мерчом\nВерим, подфартит, наши постели портативны\nМенестрелю два пути: корпоратив или квартирник\nСхемы однотипны, все теперь MC\nВедь, смену породив, мы здесь достигли смены парадигмы\nТеперь рэп — многопартийный; бэттлов наплодив\nЯ смотрю в зеркало по типу: «Сколько бед наворотил ты!»\nЯ б весь рэп поработил, но всё время в пути\nУ индустрии нервный тик, валокордин — стенокардийным\nСоберите суд, но победителей не судят\nМы первые кроманьонцы — мы выбились в люди\nНе пизди! Я кладу на вас, челядь, пятикратно\nВедь мы выступаем сильно, будто челюсть питекантропа\nВесь мой рэп, если коротко, про то, что\nУж который год который город под подошвой\nВ гору, когда прёт; потом под гору, когда тошно\nЯ не то, чтоб Гулливер, но всё же город под подошвой\nГород под подошвой, город под

In [68]:
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write('\n'.join(dataset["train"]["text"]) + '\n')

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=32)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Загрузка модели

In [82]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to('cuda:0')

In [70]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.0,
          max_length=300,
          repetition_penalty=3.0,
      )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

### Проверка модели без дообучения

In [83]:
generate("Год назад я сидел на скамейке в общественном парке")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Год назад я сидел на скамейке в общественном парке, а около меня стояла девушка.  Она была одета в короткую юбку и кофту с длинными рукавами.  На ней было розовое пальто, которое подчеркивало ее стройную фигуру.  В руке она держала букет белых лилий. 
 - Привет -- поздоровалась она со мной. 
 - Привет -- ответил я. 
 - Ты не против, если я тебя представлю своим друзьям 
 - Да нет, почему бы и нет -- ответил я. 
 - Я Вика.  А это мой друг Сергей. 
 Мы пожали друг другу руки. 
 - Пойдем к нам! -- предложила Вика. 
 По пути домой мы говорили о том, что произошло прошлой ночью.  Было очень интересно узнать все подробности.  Когда мы подошли к нашему подъезду, то зашли в квартиру.  Как только дверь за нами закрылась, я сразу же почувствовал сильный запах духов.  Это был какой-то странный запах.  От него у меня закружилась голова.  Я подошел к шкафу и начал искать дезодорант.  Но его нигде не было.  Тогда я достал пачку сигарет и закурил.  Через некоторое время из шкафа до меня донесся знако

*Как можно заметить, текст, сгенерированный моделью gpt без finetuning, является абстрактной прозой, и не сопадает со стилем исполнителя*

## Дообучение модели

In [73]:
training_args = TrainingArguments(
    output_dir="./finetuned",         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=10,              # number of training epochs
    per_device_train_batch_size=20,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5,
    seed=42  # set the random seed here
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

In [74]:
trainer.train()

Step,Training Loss


Step,Training Loss
500,3.988400


TrainOutput(global_step=620, training_loss=3.941444150863155, metrics={'train_runtime': 1404.725, 'train_samples_per_second': 44.329, 'train_steps_per_second': 0.441, 'total_flos': 3591924067467264.0, 'train_loss': 3.941444150863155, 'epoch': 9.94})

In [75]:
model.save_pretrained('model_oxxxy1')

## Проверка модели после дообучения

In [76]:
generate("Год назад я сидел на скамейке в общественном парке")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Год назад я сидел на скамейке в общественном парке, и мне говорили: «Будь мужиком!»
А сегодня я стою у памятника Неизвестному Солдату
Я гляжу через плечо, вижу, как ты ждёшь меня под дождём
И всё это время наблюдал за мной с той скамейки
Ты знаешь, что я знаю? Что я не просто так стоял там
На этом месте до сих пор стоит памятник безымянному солдату
Неизвестный Солдат
Мёртвый солдат
Неведома зверюга
Дождь моросит, лужи по колено
Мне нечего надеть, ни денег, ни даже зонта
Но я верю, что смогу пройти свой путь
Ведь каждый мой шаг приближает меня к цели
Когда-нибудь он дойдёт до пункта назначения
Мой путь уже почти близок
Впереди только цель
Пройдёт ещё много лет
Всё это время я буду смотреть сквозь слёзы
Видеть твой силуэт вдалеке
Где-то вдали раздаётся вой полицейской сирены
Это значит, что скоро мы увидимся снова
У тебя будет новый бойфренд
Твоя жена ждёт ребёнка
Слёзы градом катятся по моим щекам
Клянусь, я сделаю всё, чтобы доказать ей, что она ошибается
Моя жизнь — череда сплошных ош

Как можно заметить, после дообучения тексты не имеют рифмы, однако структурой похожи на тексты песен. А самое главное, при одинаковом запросе тексты после fintuning стали напоминать стиль оксимирона.
К сожалению, обучение 10 эпох занимает 20 минут, поэтому я не стал делать больше. Скорее всего, при большем количестве эпох можно было бы добиться лучших результатов.

*Ниже приведено еще несколько примеров:*

In [80]:
generate("Я покидаю дом в семь часов утра")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Я покидаю дом в семь часов утра
Иду на улицу, и мой путь пролегает мимо окон
Вон там, за окном, дома, дворы, улицы
Но я не сойду с места до тех пор, пока не доберусь до цели
До того самого окна, где отражается город
Окна домов похожи друг на друга как две капли воды
Они все разные, но у них есть общие черты
У каждой — своя история, свои герои
Моя жизнь полна сложностей, они меня пугают
Мне страшно представить, что со мной будет, когда я достигну цели
Пока я жив, я буду пытаться дойти до цели
Мой путь тернист, но он всегда приводит к цели
Ведь я иду по нему уже больше десяти лет
За это время столько всего произошло
Так почему бы мне не рассказать вам о своих приключениях?
Начнём…
Давным-давно, когда мир был ещё безгранично прост
Поздним вечером мы с моим другом шли домой
Мы брели по ночной пустынной улице
Извилистый путь привёл нас к заброшенному дому
Домик из серого камня, заросший плющом
Мимо проносятся тёмные силуэты зданий
Вдруг откуда ни возьмись появляется странный силуэт
Он машет

In [78]:
generate("Всё переплетено, море нитей, но")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Всё переплетено, море нитей, но всё равно тесно
И так каждый год по кругу
Я не верю в реинкарнацию — это бредни дилетанта
Не надо мне сказок про переселение душ
Коль мы живём на этой земле, то должны быть едины
Все вместе и каждый за себя
Мы все умрём, но пока эта земля наша
Каждый из нас с ней неразлучен
Вокруг нас океан, где-то вдалеке маяк
Он светится, словно далёкий маяк
Где-то там, вдали, что-то вспыхнуло
Но оно погасло, как будто спичка
Зажжённая неверным движением руки
Ничто не вечно под луной
Лишь наши сердца бьются в унисон
Слышишь, сердце? Оно бъётся у горла
А ну-ка, пой! Не бойся, тише, тише, тише!
Это пульс земли, она слышит нас
Гул прибоя, он говорит нам: «Тихо!»
Зови меня своим именем, зовите меня своим именем
Только моим именем, зовите меня своим именем
Всеми своими именами, зовите меня своим именем
Моим именем, зовите меня своим именем
Останусь здесь, останусь здесь, останусь здесь, останусь здесь
Остался тут, остался тут, остался тут, остался тут, остался тут
Остался т